# Ion exchange in dune sand and porewater

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Mar 10th, 2022</p>

With this tutorial, we model the ion exchange of the Dune sand and porewater. We consider 6 g of Dune sand with the cation exchange capacity CEC = 10 meq/kg, which results in 0.06 mol of the exchanger X (60 meq/L, default 1L).

We aim to reproduce the results of the PHREEQC script below.

```text
DATABASE ../database/phreeqc.dat
TITLE Exchange in Dune sand

SOLUTION 1
pressure 1.0
temperature 25.0
Na 1.10006
Mg 0.48
Ca 1.9

EXCHANGE 1
X 0.06	#moles
-equilibrate 1
```

We start with setting up the chemical system and other means of modelling ion exchange in Reaktoro framework. For that, we use the PHREEQC database `phreeqc.dat` containing ion exchange species, their corresponding reactions, and necessary parameters.

In [1]:
from reaktoro import *
import numpy as np

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O C Ca Na Mg Cl"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

The ion exchange phase is modelled by the {{IonExchangePhase}} class. We select only three ion exchange species, i.e., NaX, CaX{{_2}}, and MgX{{_2}}, that can be formed based on the porewater content. We use the Gaines-Thomas activity model (considering equivalence fractions as activities for NaX, CaX{{_2}}, and MgX{{_2}}).

In [2]:
exchange = IonExchangePhase("NaX CaX2 MgX2")
exchange.setActivityModel(ActivityModelIonExchangeGainesThomas())

system = ChemicalSystem(db, solution, exchange)

# Define the equilibrium solver
solver = EquilibriumSolver(system)

Below, we define the initial chemical state for the equilibrium ion exchange simulations of the Dune sand and porewater for the fixed temperature and pressure.

```{note}
We note that the solution is scaled by 1e6 to match the equilibration of the exchanger happening in the PHREEQC script.
This way we imitate an unlimited source of brine in the equilibrium calculations.
```

```{note}
The current implementation of the ion exchange assumes that the exchanger (X, in this case) is always occupied by some ions.
As the default ion, we consider Na<sup>+</sup> as all the `logk` parameters of exchange species in `phreeqc.dat` are
calculated in relation to Na<sup>+</sup> + X<sup>-</sup> = NaX reaction.
```

In [3]:
# Define temperature and pressure
T = 25.0 # temperature in celsius
P = 1.0  # pressure in bar

# Define initial equilibrium state
state = ChemicalState(system)
state.setTemperature(T, "celsius")
state.setPressure(P, "bar")
# Scale solution recipe to match the values of the PHREEQC examples
state.setSpeciesMass("H2O"   , 1.e6, "kg")
state.setSpeciesAmount("Na+" , 1.10, "kmol")
state.setSpeciesAmount("Mg+2", 0.48, "kmol")
state.setSpeciesAmount("Ca+2", 1.90, "kmol")
# Set the number of exchange assuming that it is completely occupied by sodium
state.setSpeciesAmount("NaX" , 0.06, "mol")

We expect the following set of ion exchange reactions happening between porewater and exchanger site NaX:

$$
\begin{alignat}{4}
\tfrac{1}{2} {\rm Ca^{+2}} + {\rm NaX} & \rightleftharpoons {\rm Na}^+ + \tfrac{1}{2} {\rm CaX}_2\\
\tfrac{1}{2} {\rm Mg^{+2}} + {\rm NaX} & \rightleftharpoons {\rm Na}^+ + \tfrac{1}{2} {\rm MgX}_2\\
\end{alignat}
$$

After the chemical state is provided, we can define the equilibrium solver and run the Gibbs energy minimization with the given initial state.

In [4]:
solver = EquilibriumSolver(system)
solver.solve(state)

The properties of resulting chemical state can be inspected by printing the state itself:

In [5]:
print(state)

+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   298.1500 |    K |
| Pressure        |     1.0000 |  bar |
| Charge:         | 5.8600e+03 |  mol |
| Element Amount: |            |      |
| :: X            | 6.0000e-02 |  mol |
| :: H            | 1.1101e+08 |  mol |
| :: C            | 1.2000e-15 |  mol |
| :: O            | 5.5506e+07 |  mol |
| :: Na           | 1.1001e+03 |  mol |
| :: Mg           | 4.8000e+02 |  mol |
| :: Cl           | 1.0000e-16 |  mol |
| :: Ca           | 1.9000e+03 |  mol |
| Species Amount: |            |      |
| :: CO3-2        | 1.0000e-16 |  mol |
| :: H+           | 1.1653e-01 |  mol |
| :: H2O          | 5.5506e+07 |  mol |
| :: CO2          | 1.0000e-16 |  mol |
| :: (CO2)2       | 1.0000e-16 |  mol |
| :: HCO3-        | 1.0000e-16 |  mol |
| :: CH4          | 1.0000e-16 |  mol |
| :: Ca+2         | 1.9000e+03 |  mol |
| :: CaCO3        | 1.0000e-16 |  mol |


Additionally, we can study the properties of the brine after the ion exchange has been performed. For this purpose, the {{AqueousProps}} class can be used.

In [6]:
aqprops = AqueousProps(state)
print("I  = %f mol/kgw" % float(aqprops.ionicStrength()))
print("pH = %f" % float(aqprops.pH()))
print("pE = %f" % float(aqprops.pE()))


I  = 0.005310 mol/kgw
pH = 6.966945
pE = -3.999323


To evaluate the ion exchange properties of species NaX, CaX{{_2}}, and MgX{{_2}}, the class {{IonExchangeProps}} can be used. It can be initialized via the instance of the {{ChemicalState}} class.

In [7]:
exprops = IonExchangeProps(state)
print(exprops)

+--------------------------------------+------------+------+
| Property                             |      Value | Unit |
+--------------------------------------+------------+------+
| Element Amounts:                     |            |      |
| :: X                                 | 6.0000e-02 |  mol |
| :: Na                                | 5.5828e-04 |  mol |
| :: Mg                                | 4.0703e-03 |  mol |
| :: Ca                                | 2.5651e-02 |  mol |
| Species Amounts:                     |            |      |
| :: NaX                               | 5.5828e-04 |  mol |
| :: CaX2                              | 2.5651e-02 |  mol |
| :: MgX2                              | 4.0703e-03 |  mol |
| Equivalents:                         |            |      |
| :: NaX                               |     0.0006 |   eq |
| :: CaX2                              |     0.0513 |   eq |
| :: MgX2                              |     0.0081 |   eq |
| Equivalent Fractions: 